# The tool to visualize lava flows, burnfires with Sentinel-2 multispectral imagery
The tool allows to enhance the visualization of "high-temperature areas" within the different background: from Natural Colors to different combintations of SWIR bands. Also, the possibility to blend several band visualizations is supported. The choice of all parameters (sensitivity, saturation, manual adjustment, stretching) could be done manually. The full description of all functionality with step-by-step tutorial could be found on github repositorium. Finally, the visualization of final image could be corrected.

Fix the bug with visualization of area of interest

In [1]:
# Initializing for the use of Earth Engine services
# In case of the first time use you should authenticate by pasting token in the box and then pressing enter
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [2]:
import geemap
import functools
import itertools
import ipywidgets as widgets
from ipyleaflet import (WidgetControl, Map)
import datetime
import numpy as np
import colorama
from colorama import Fore

---
* **Select the area of interest**

Choose the area of interest on the map or indicate the coordinates and buffer range (the circle-shape area is not supported, select between rectangle or polygon)
In order to define your own area of interest click box `User-defined AOI` and then select the on the map with embedded instruments on the left panel

In [14]:
style = {'description_width': 'initial'}

lat_widget = widgets.FloatText(description='Latitude:', max_height=20,  style=style, layout=widgets.Layout(width='25%'))
long_widget = widgets.FloatText(description='Longtitude:', max_height=20,  style=style, layout=widgets.Layout(width='25%'))
aoi_widget = widgets.Checkbox(value=False, description='User-defined AOI', disabled=False)
buffer_widget = widgets.IntText(description = 'Distance:', layout=widgets.Layout(width='35%'))

def aoi_change(change):
    m.layers = m.layers[:3] 
    m.user_roi = None
    m.user_rois = None
    m.draw_count = 0
    lat_widget.value = 0
    long_widget.value = 0
    buffer_widget.value = 0
    # output_widget.clear_output()
    
aoi_widget.observe(aoi_change, names='value')

coord_widgets = widgets.HBox([lat_widget, long_widget, aoi_widget])
coord_buffer = widgets.VBox([coord_widgets, buffer_widget])
display(coord_buffer)

* **Select the date range**

After the area of interest has been chosen the next step is to define the interval. Choose the range of dates that you are interested in and then click the button `List images`. After a while a list with all available dates and associated images will appear under these widgets. In case you want to change parameters you could just fill new values to the fields with coordinates or just click `Clear fields`

In [4]:
submit = widgets.Button(description='List images', button_style='primary', tooltip='Click to submit', style=style)
clear_fields = widgets.Button(description='Clear fields', button_style='warning', tooltip='Click to clear', style=style)
start_day = widgets.DatePicker(description='Start date: ', disabled=False)
end_day = widgets.DatePicker(description='End date: ', disabled=False)
output_widget_listimages = widgets.Output()

widget_buttons = widgets.HBox([submit, clear_fields])
widget_dates = widgets.HBox([start_day, end_day])
final_widget = widgets.VBox([widget_dates, widget_buttons, output_widget_listimages])
display(final_widget)

* **Select the image**

After the date list is appeared on your screen you should choose one of your interest for further visualization. After that type its number to the field below and click `Submit`. If everything is correct the message *"The image is selected"* will appear on the screen. 

In [5]:
day_select_widget = widgets.IntText(value=1, description='Date no.:', disabled=False)
submit_sel = widgets.Button(description='Submit', button_style='primary', tooltip='Click to submit', style=style)
output_widget_img = widgets.Output()
display(widgets.HBox([day_select_widget, submit_sel]))
display(output_widget_img)

Output()

In [6]:
m = geemap.Map()
m.add_basemap('TERRAIN')
m.addLayerControl()
output_widget = widgets.Output()
output_control = WidgetControl(widget=output_widget, position='bottomright')
m.add_control(output_control)
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [13]:
def on_list_clicked(b):
    with output_widget_listimages:
        output_widget_listimages.clear_output()
        print("Listing images...")
        
        try:
            use_aoi = aoi_widget.value

            if use_aoi:
                if m.user_roi is not None:
                    global roi
                    roi = m.user_roi
                    layer_name = 'User drawn AOI'
                    point = roi.centroid(maxError = 2)
                else:
                    output_widget_listimages.clear_output()
                    print('No user AOI is defined')
                    return
            else:
                x_coords = long_widget.value
                y_coords = lat_widget.value
                buffer_dist = buffer_widget.value
                point = ee.Geometry.Point([x_coords, y_coords])
                buffer = point.buffer(int(buffer_dist))
                roi = buffer.bounds()
                layer_name = 'AOI from coordinates'
                m.addLayer(roi, {'color': 'blue'}, 'Area of interest')
            
            m.addLayer(ee.Image().paint(roi, 0, 2), {'palette': 'red'}, layer_name)
            start = ee.Date(start_day.value.strftime('%Y-%m-%d'))
            end = ee.Date(end_day.value.strftime('%Y-%m-%d'))

            zoom = 10
            m.centerObject(point, zoom)

            global collection
            collection = collection_retr(start, end, point)
            imagelist = collection.toList(collection.size())

            allDates = ee.List(collection.aggregate_array('system:time_start')).getInfo();

            def date_retr(date):
                return ee.Date(date).format('YYYY-MM-dd').getInfo()

            allDatesSimple = list((map(date_retr, allDates)))
            allDatesSimple = np.array(allDatesSimple)

            output_widget_listimages.clear_output()
            global list_dates
            list_dates = np.unique(allDatesSimple)
            for num, date in enumerate(list_dates):
                print(str(num+1) + ') ' + date)
                
        except Exception as e:
            print(e)
            print(Fore.RED + 'Try to select other parameters. An error occured!')

            
               
submit.on_click(on_list_clicked)

def clear_fields_clicked(b):
    try:
        lat_widget.value = 0
        m.layers = m.layers[:3]  
        long_widget.value = 0 
        aoi_widget.value = False
        buffer_widget.value = 0
        start_day.value = None
        end_day.value = None
    except Exception as e:
        with output_widget_listimages:
            output_widget_listimages.clear_output()
            print(Fore.RED + "An unknown error occured.")
            print(e)
        

clear_fields.on_click(clear_fields_clicked)

In [8]:
def on_submit_clicked(b):
    with output_widget_img:
        output_widget_img.clear_output()
        
        try:
            day = list_dates[day_select_widget.value-1]
            day_end = day + 'T23:59:59'
            day_start = ee.Date(day)
            day_end = ee.Date(day_end)
            selected_images = collection.filterDate(day_start, day_end)
            global image_final
            image_final = selected_images.first()
            print("The image is selected!")
        except Exception as e:
            print(e)
            print(Fore.RED + 'Try to select again the number from the list. An error occured!')

submit_sel.on_click(on_submit_clicked)

* **Define the visualization parameters in the panel below**

Firstly, select the band combinations to visulaize. You could blend two options by changing their opacity with a slider under each dropdown window. Furthermore, stretching parameters for the initial image could be modiefied by varying minimum and maximum stretching values. Other parameters are saturation of the image and sensitivity of fire detection.
After all, there is a manual adjustment of the final image and visualization parameters: minimum and maximum values.

In [9]:
visbands = widgets.Dropdown(description='Select band combination 1:',
                                  options=['Natural Colors', 'Enhanced Natural Colors', 'NIR SWIR Colors 1', 'NIR SWIR Colors 2', 'NIR SWIR Colors 3', 'NIR SWIR Colors 4', 'Natural NIR SWIR Mix', 'False Color', 'Natural False Color',
                                          'Vegetation', 'Pan Band'], value = 'NIR SWIR Colors 2', style=style, layout=widgets.Layout(width='30%'))
visbands_2 = widgets.Dropdown(description='Select band for combination 2:',
                                  options=['Natural Colors', 'Enhanced Natural Colors', 'NIR SWIR Colors 1', 'NIR SWIR Colors 2', 'NIR SWIR Colors 3', 'NIR SWIR Colors 4', 'Natural NIR SWIR Mix', 'False Color', 'Natural False Color',
                                          'Vegetation', 'Pan Band'], value = 'Enhanced Natural Colors', style=style, layout=widgets.Layout(width='30%'))
opac_1 = widgets.FloatSlider(description = 'Opacity 1:', value = 0, min = 0, max = 1, step = 0.05, layout=widgets.Layout(width='30%'))
opac_2 = widgets.FloatSlider(description = 'Opacity 2:', value = 1, min = 0, max = 1, step = 0.05, layout=widgets.Layout(width='30%'))
visbands_widgets = widgets.HBox([visbands, visbands_2])
opac_widget = widgets.HBox([opac_1, opac_2])


stretch_min = widgets.FloatText(description = 'Stretch min:', step = 0.05, value = 0.01, layout=widgets.Layout(width='12.5%'))
stretch_max = widgets.FloatText(description = 'Stretch max:', width = '20%', step = 0.05, value = 0.9, layout=widgets.Layout(width='12.5%'))
satur = widgets.FloatText(description = 'Saturation:', width = '20%', step = 0.05, value = 1, layout=widgets.Layout(width='12.5%'))
sensit = widgets.FloatText(description = 'Fire sensitivity:', width = '20%', step = 0.05, value = 1, layout=widgets.Layout(width='12.5%'))
stretch_widget = widgets.HBox([stretch_min, stretch_max, satur, sensit])

corr_R = widgets.FloatText(value = 0, description = 'Adjust R:', step = 0.05, layout=widgets.Layout(width='16%'))
corr_G = widgets.FloatText(value = 0, description = 'G:', width = '20%', step = 0.05, layout=widgets.Layout(width='16%'))
corr_B = widgets.FloatText(value = 0, description = 'B:', width = '20%', step = 0.05, layout=widgets.Layout(width='16%'))
correction_widget = widgets.HBox([corr_R, corr_G, corr_B])
cropping_widget = widgets.RadioButtons(options=['Crop ROI', 'Show full image'], description='Crop area:', value = 'Show full image', disabled=False)
visParams_widget_1 = widgets.FloatText(description = "Vis. min:", value = 0.0, step = 0.1, max = 5,  layout=widgets.Layout(width='15%'))
visParams_widget_2 = widgets.FloatText(description = "max:", value = 0.8, step = 0.1, max = 5,  layout=widgets.Layout(width='15%'))
visParams_widget = widgets.HBox([cropping_widget, visParams_widget_1, visParams_widget_2])

vis = widgets.Button(description='Visualize', button_style='primary', tooltip='Click to visualize', style=style)
clear_map = widgets.Button(description='Clear map', button_style='warning', tooltip='Click to clear', style=style) #set up clear map
widgets_mapping = widgets.HBox([vis, clear_map])
visual_widget = widgets.VBox([widgets.Label('Visualization selection'), visbands_widgets, opac_widget, 
                              widgets.Label('Image and fire detection adjustment'), stretch_widget, correction_widget, 
                              widgets.Label('Cropping and visalization'), visParams_widget, widgets_mapping])
display(visual_widget)

In [10]:
def clear_map_clicked(b):
        m.clear_layers()
        m.add_basemap('TERRAIN')
clear_map.on_click(clear_map_clicked)

In [11]:
def visualize(b):
    with output_widget:
        output_widget.clear_output()
        print('Visualazing...')
        Map.default_style = {'cursor': 'wait'}
        
        try:
            image = image_final.divide(10000)

            def stretch(image, min, max):
                return (image.subtract(min)).divide(max-min)

            Fire1OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]
            Fire2OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99).add(0.25), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]
            Fire3OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99).add(0.5), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]

            fire_layers = [ee.Image(Fire1OVL), ee.Image(Fire2OVL), ee.Image(Fire3OVL)]

            def combo_selection(input, image):
                if input == 'Natural Colors':
                    layer = [image.select('B4').multiply(2.9), image.select('B3').multiply(3.1), image.select('B2').multiply(3.0)] #NatruralColors
                elif input == 'Enhanced Natural Colors':
                    layer = [(image.select('B4').multiply(2.8)).add(image.select('B5').multiply(0.1)), (image.select('B3').multiply(2.8)).add(image.select('B8').multiply(0.15)),
                                 image.select('B2').multiply(2.8)] #Enhanced Natural Colors
                elif input == 'NIR SWIR Colors 1':
                    layer = [image.select('B12').multiply(2.6), image.select('B8').multiply(1.9), image.select('B2').multiply(2.7)] #NIRSWIRColors1
                elif input == 'NIR SWIR Colors 2':
                    layer = [image.select('B12').multiply(2.4), image.select('B8A').multiply(1.7), image.select('B5').multiply(2.2)] #NIRSWIRColors2
                elif input == 'NIR SWIR Colors 3':
                    layer = [((image.select('B12').add(image.select('B11'))).multiply(0.5).divide(4)).divide(image.select('B7')), image.select('B8A').multiply(0.8), image.select('B7')] #NIRSWIRColors3
                elif input == 'NIR SWIR Colors 4':
                    layer = [image.select('B12').multiply(2.0), image.select('B11').multiply(1.1), image.select('B8').multiply(1.6)] #NIRSWIRColors4
                elif input == 'Natural NIR SWIR Mix':
                    layer = [(image.select('B4').multiply(2.1)).add(image.select('B12').multiply(0.5)), (image.select('B3').multiply(2.2)).add(image.select('B8').multiply(0.5)),
                             image.select('B2').multiply(3.0)] #NaturalNIRSWIRMix
                elif input == 'False Color':
                    layer = [image.select('B8').multiply(2), image.select('B4').multiply(2), image.select('B3').multiply(2)] #FalseColor
                elif input == 'Natural False Color':
                    layer = [image.select('B12').multiply(2.6), image.select('B11').multiply(2), image.select('B4').multiply(2.7)] #NatFalseColor
                elif input == 'Vegetation':
                    layer = [image.select('B11').multiply(2.4), image.select('B8A').multiply(2), image.select('B4').multiply(2.9)] #Vegetation
                elif input == 'Pan Band':
                    layer = [image.select('B8'), image.select('B8'), image.select('B8')] #PanBand
                return layer

            layer1 = combo_selection(visbands.value, image)
            layer2 = combo_selection(visbands_2.value, image)
            layer1Amount = opac_1.value
            layer2Amount = opac_2.value

            stretchMin = stretch_min.value
            stretchMax = stretch_max.value
            saturation = satur.value
            fireSensitivity = sensit.value
            manualCorrection = [corr_R.value, corr_G.value, corr_B.value]

            def blend(bArr1, bArr2, opa1, opa2):
                result_list =[]
                for (a_img,b_img) in zip(bArr1, bArr2):
                    result_list.append(a_img.divide(opa1).add(b_img.divide(opa2)))
                return result_list

            noFire = blend(layer1, layer2, layer1Amount, layer2Amount)

            def satEnh(rgbArr):
                avg = functools.reduce(lambda a, b: a.add(b), rgbArr).divide(len(rgbArr))
                result = [(avg.multiply(1 - saturation)).add(img.multiply(saturation)) for img in rgbArr]
                return result 

            def applyEnh(bArr):
                return satEnh([stretch(bArr[0], stretchMin, stretchMax), stretch(bArr[1], stretchMin, stretchMax), stretch(bArr[2], stretchMin, stretchMax)])

            def correction(manual_num, arrRGB):
                return [arrRGB[0].add(manual_num[0]), arrRGB[1].add(manual_num[1]), arrRGB[2].add(manual_num[2])]

            noFire_final = applyEnh(noFire)
            noFire_final = correction(manualCorrection, noFire_final)
            noFire_final = ee.Image(noFire_final).rename(['R_new', 'G_new', 'B_new'])

            img_exp = image.select('B11').add(image.select('B12')).rename('SWIR_dif') 

            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(1/fireSensitivity), fire_layers[0])
            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(2/fireSensitivity), fire_layers[2])
            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(1.5/fireSensitivity), fire_layers[1])

            visParams = {"min": visParams_widget_1.value, "max": visParams_widget_2.value, "bands": ["R_new", "G_new", "B_new"]}

            date = ee.Date(image_final.get('system:time_start')).format('YYYY-MM-dd')
            
            if cropping_widget.value == 'Crop ROI':   
                m.addLayer(noFire_final.clip(roi), visParams,  visbands.value + ' ' + visbands_2.value + '; ' + date.getInfo())
            else:
                m.addLayer(noFire_final, visParams, visbands.value + ' ' + visbands_2.value + '; ' + date.getInfo())
            output_widget.clear_output()
            
        except Exception as e:
            print(e)
            print(Fore.RED + 'Try again! An error occured')
            
vis.on_click(visualize)

In [12]:
def collection_retr(start, finish, roi):
    collection = "COPERNICUS/S2" 
    s2_coll = (ee.ImageCollection(collection)
        .filterDate(start, finish)
        .filterBounds(roi))
    return s2_coll

In [15]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

ipywidgets==7.7.1
numpy==1.23.1
ipyleaflet==0.17.0
geemap==0.16.1
colorama==0.4.5
